In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import BertTokenizer, TFBertModel, BertConfig, BertTokenizerFast 
print(tf.__version__)

2.8.0


In [ ]:
data = pd.read_csv("/content/drive/My Drive/tamil_data_complete.tsv",sep="\t")
test = pd.read_csv("/content/drive/My Drive/test_data_final.tsv",sep="\t")

In [ ]:
data.head()

,spans,text
0,Correct. Enga apa military da oodi vilaiyada s...,"[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4..."
1,Dei Rajini pavam da ne varaven poraven ellam k...,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3..."
2,"Dey dey deyyy,, loosu pasangala,, munna pinna ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Intha maari comments ku like kekuravangala ind...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
4,250 k likes ineram sila arivuketta pundaika it...,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."


In [ ]:
# Span BERT
# 200, 300 and 600
max_len = 500

tokenizer = BertTokenizerFast.from_pretrained("SpanBERT/spanbert-base-cased")

In [ ]:
# Evaluation metric

import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval

def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


def evaluate(pred, gold):
    """
    Based on https://github.com/felipebravom/EmoInt/blob/master/codalab/scoring_program/evaluation.py
    :param pred: file with predictions
    :param gold: file with ground truth
    :return:
    """
    # # read the predictions
    # pred_lines = pred.readlines()
    # # read the ground truth
    # gold_lines = gold.readlines()

    pred_lines = pred
    gold_lines = gold

    # only when the same number of lines exists
    if (len(pred_lines) == len(gold_lines)):
        data_dic = {}
        for n, line in enumerate(gold_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                data_dic[int(parts[0])] = [literal_eval(parts[1])]
            else:
                raise ValueError('Format problem for gold line %d.', n)

        for n, line in enumerate(pred_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(literal_eval(parts[1]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append([])
                else:
                    raise ValueError('Invalid text id for pred line %d.', n)
            else:
                raise ValueError('Format problem for pred line %d.', n)

        # lists storing gold and prediction scores
        scores = []
        for id in data_dic:
            if len(data_dic[id]) == 2:
                gold_spans = data_dic[id][0]
                pred_spans = data_dic[id][1]
                scores.append(f1(pred_spans, gold_spans))
            else:
                sys.exit('Repeated id in test data.')

        return (np.mean(scores), sem(scores))

In [ ]:
## To retrive the toxic words from the given input file

def span_retrived(text_data, spans):
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(retrieve_word_from_span(spans[i], text_data[i]))
    
    return token_labels


In [ ]:
## for retriving toxic word for every sample

def retrieve_word_from_span(lst_span, text):
    i = 0
    token = []
    a = 0

    word = []

    while (i < (len(lst_span) - 1)):
        if (lst_span[i] != (lst_span[i+1]-1)):
            token.append(lst_span[a:(i+1)])
            a = i + 1
        elif i == (len(lst_span) - 2):
            token.append(lst_span[a:i+2])

        i = i + 1

    for t in token:
        word.append(text[t[0]:(t[len(t)-1])+1])

    return word


In [ ]:
def span_convert(text_data, spans):
    MAX_LEN = 200
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(retrieve_word_from_span(spans[i], text_data[i]))

    lst_seq = []
    for i in range(0, len(text_data)):
        # token = tknzr.tokenize(text_data[i])
        token = custom_tokenizer(text_data[i])
        if len(token) > MAX_LEN:
            MAX_LEN = len(token)
            
        seq = np.zeros(len(token), dtype=int)
        for j in range(0, len(token)):
            for t in token_labels[i]:
                # if token[j] in tknzr.tokenize(t):
                if token[j] in custom_tokenizer(t):
                    seq[j] = 1
        lst_seq.append(seq)     

    return (token_labels, lst_seq)

In [ ]:
def custom_tokenizer(text_data):
    return tokenizer.tokenize(text_data)

In [ ]:
from ast import literal_eval

spans = data['text'].apply(literal_eval)

In [ ]:
data['token'], data['seq'] = span_convert(data['spans'].values, spans)

In [ ]:
data.head()

,spans,text,token,seq
0,Correct. Enga apa military da oodi vilaiyada s...,"[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4...",[oodi vilaiyada solli tharuvaaaayaa],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ..."
1,Dei Rajini pavam da ne varaven poraven ellam k...,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",[varaven poraven ellam karithuppatha koraiya p...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,"Dey dey deyyy,, loosu pasangala,, munna pinna ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Dey dey deyyy,, loosu pasangala,, pathruking...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Intha maari comments ku like kekuravangala ind...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5...",[ india va vittu veliya annupanum],"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,250 k likes ineram sila arivuketta pundaika it...,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[arivuketta pundaika ithku karanam ],"[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# start index, end index
# selected_text -- tokenized text

# Fun - preprossin
# text = Correct. Enga apa military da oodi vilaiyada s
# start_char_index = 30
# selected text = oodi vilaiyada solli tharuvaaaayaa - toxic words
# all_answers = predicted text will be saved

class TrainTweetExample:
    def __init__(self,  text, start_char_idx, selected_text, all_answers):
        # self.sentiment = sentiment
        self.text = text
        self.start_char_idx = start_char_idx
        self.selected_text = selected_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        text = self.text
        # sentiment = self.sentiment
        selected_text = self.selected_text
        start_char_idx = self.start_char_idx

        # Clean text, sentiment, and answer
        # sentiment = " ".join(str(sentiment).split())
        text = " ".join(str(text).split())
        answer = " ".join(str(selected_text).split())

        # Find end character index of answer in text
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(text):
            self.skip = True
            return

        # Mark the character indexes in text that are in answer
        is_char_in_ans = [0] * len(text)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize text
        tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = max_len)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_text.offset_mapping):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize sentiment
        # tokenized_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True, max_length = max_len)

        # Create inputs
        input_ids = tokenized_text.input_ids 
        token_type_ids = [0] * len(tokenized_text.input_ids)
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_text.offset_mapping

In [ ]:
## this function will created our dataset which will be input to the model . The object of class TrainTweet Examples is used to create dataset.

def create_tweet_examples(data):
    tweet_examples = []
    all_answers = data.token.values
    count = 0
    for index, row in data.iterrows():
        # sentiment = row['sentiment']
        text = row['spans']
        selected_text = row['token']
        # print(row.text)
        start_char_idx = row.spans.index(row.token[0])
        tweet_eg = TrainTweetExample(
              text, start_char_idx, selected_text, all_answers
        )
        tweet_eg.preprocess()
        tweet_examples.append(tweet_eg)
        # except: ## keep track of the number that can't be tokenized/processed
        #     count += 1 
    # print("Couldn't process",count,"points")
    return tweet_examples

In [ ]:
# The ouput generated by the create_tweet_examples will be input to given function.

# x = input_ids, token_type_ids and attention_mask which are defined in Train_Tweet_example
# y = start_token_idx, end_token_idx

# input_ids = ids created by the tokanizer
# token_type_ids = give 0 attention to this non toxic words
# attention_mast = give 1 attention to toxic words


def create_inputs_targets(tweet_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in tweet_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    print(dataset_dict["input_ids"])
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

In [ ]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(data, test_size = 0.1,random_state=10)

In [ ]:

# = y = [30,40] starting = 30,ending = 40
train_tweet_examples = create_tweet_examples(train)
x_train, y_train = create_inputs_targets(train_tweet_examples)
print(f"{len(train_tweet_examples)} training points created.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[  101 16664  1161 ...     0     0     0]
 [  101   188  2093 ...     0     0     0]
 [  101 23015  1161 ...     0     0     0]
 ...
 [  101  5358  1182 ...     0     0     0]
 [  101   180  4867 ...     0     0     0]
 [  101  4035  1605 ...     0     0     0]]
4334 training points created.


In [ ]:
np.array(x_train).shape

(3, 2312, 500)

In [ ]:
def create_model():

    PATH = 'SpanBERT/spanbert-base-cased'
    encoder = TFBertModel.from_pretrained(PATH, from_pt = True)
    
## QA Model
    input_ids = layers.Input(shape=(max_len ,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
         input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]
    

    start_logits = layers.Dropout(0.3)(embedding)
    start_logits = layers.Conv1D(128,2,padding='same')(start_logits)
    start_logits = layers.LeakyReLU()(start_logits)
    start_logits = layers.Conv1D(64,2,padding='same')(start_logits)
    start_logits = layers.Dense(1, name="start_logit")(start_logits)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dropout(0.3)(embedding)
    end_logits = layers.Conv1D(128,2,padding='same')(end_logits)
    end_logits = layers.LeakyReLU()(end_logits)
    end_logits = layers.Conv1D(64,2,padding='same')(end_logits)
    end_logits = layers.Dense(1, name="end_logit")(end_logits)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss],metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

Downloading:   0%|          | 0.00/205M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',            

In [ ]:
batch_size = 8
div = len(x_train[0]) - (len(x_train[0]) % batch_size)
x = list(np.array(x_train)[:,:div]) ## inputs must be divisible by batch size 
y = list(np.array(y_train)[:,:div])

In [ ]:
model.fit(
    list(np.array(x)),
    list(np.array(y)),
    epochs=1,
    batch_size=batch_size,
    verbose=1
)

289/289 [==============================] - 536s 2s/step - loss: 4.3269 - activation_loss: 1.6745 - activation_1_loss: 2.6525 - activation_accuracy: 0.4567 - activation_1_accuracy: 0.2522


In [ ]:
eval_tweet_examples = create_tweet_examples(validation)
x_eval, y_eval = create_inputs_targets(eval_tweet_examples)
print(f"{len(eval_tweet_examples)} evaluation points created.")

[[  101   187 12487 ...     0     0     0]
 [  101  4035  1605 ...     0     0     0]
 [  101  1126  7939 ...     0     0     0]
 ...
 [  101   187 22558 ...     0     0     0]
 [  101   185  3715 ...     0     0     0]
 [  101   178 20564 ...     0     0     0]]
482 evaluation points created.


In [ ]:
## Prediction of the test data

pred = model.predict(x_eval)

pred_start, pred_end = pred
count = 0

pred_text = []
validation['prediction'] = np.zeros(len(validation.spans.values))



In [ ]:
## Extracting the toxic words from the prediction

validation['start']=np.zeros(len(validation.spans.values))
validation['end']=np.zeros(len(validation.spans.values))


listsk = list(validation.index)
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
    tweet_eg = eval_tweet_examples[idx]
    if (tweet_eg.skip == True):
        pred_text.append("")
    else: 
        offsets = tweet_eg.context_token_to_char
        start = np.argmax(start)
        end = np.argmax(end)
        if start >= len(offsets):
            continue
        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_char_end = offsets[end][1]
            pred_ans = tweet_eg.text[pred_char_start:pred_char_end + 1]
        else:
            pred_ans = tweet_eg.text[pred_char_start:]

        validation['start'][listsk[idx]] = pred_char_start
        validation['end'][listsk[idx]] = pred_char_end

        # normalized_pred_ans = normalize_text(pred_ans)
        #print(normalized_pred_ans)
        pred_text.append(pred_ans)
        # print(pred_ans)
        validation['prediction'][listsk[idx]]= pred_ans


# for x in list(validation.index):
#   validation['start'][x] = validation['spans'][x].find(str(validation['prediction'][x]))
#   validation['end'][x] = start+len(str(validation['prediction'][x]))


## The required output i.e spans of the toxic words

lists = list()
for x in list(validation.index):
  a = np.array(range(int(validation['start'][x]),int(validation['end'][x])+1))
  lists.append(a.astype('int'))

validation['spans_predict'] = lists


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying 

In [ ]:
validation['text'] = validation['text'].apply(literal_eval)

In [ ]:
# spans = given sentence
# text = spans of toxic words
# token = extracted toxic word
# prediction = predicted toxic word
# start = predicted starting index
# end = predicted ending index

validation.head()

,spans,text,token,seq,prediction,start,end,spans_predict
1309,Rajini Ku sema mokka kuduthurukanga nice comed...,"[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2...",[mokka kuduthurukanga ],"[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",mokka kuduthurukanga nice,15.0,40.0,"[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2..."
4574,Ethuku 63k dislike apdi enna pathutinga,"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...",[apdi enna pathutinga],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]",0.0,0.0,0.0,[0]
422,Enna da saw movie copy ah,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]",[movie copy ],"[0, 0, 0, 0, 1, 1, 0]",0.0,0.0,0.0,[0]
97,Enna dhan kaalam maranalum andha sadhi veri ad...,"[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 5...","[adanga matrangu , sc vela bc Vella illa Enna ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",ranalum andha,19.0,32.0,"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3..."
1835,Final ah thalaivara vachu senjitingaleh dah,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...",[vachu senjitingaleh d],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",0.0,0.0,0.0,[0]


In [ ]:
acc = []
for i in list(validation.index):
    acc.append(f1(np.array(validation['text'][i]), np.array(validation['spans_predict'][i])))

print(np.mean(acc)*100)

10.36244180601232


In [ ]:
class TestTweetExample:
    def __init__(self,  text):
        # self.sentiment = sentiment
        self.text = text
        self.skip = False

    def preprocess(self):
        text = self.text
        # sentiment = self.sentiment

        # Clean text, answer and sentiment
        # sentiment = " ".join(str(sentiment).split())
        text = " ".join(str(text).split())

        # Tokenize text
        tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = max_len)

        # Tokenize sentiment
        # tokenized_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True, max_length = max_len)

        # Create inputs
        input_ids = tokenized_text.input_ids
        token_type_ids = [0] * len(tokenized_text.input_ids)
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.text_token_to_char = tokenized_text.offset_mapping
        self.start_token_idx = 0
        self.end_token_idx = 0

In [ ]:
def create_tweet_examples_test(data):
    tweet_examples = []
    all_answers = None
    count = 0
    for index, row in data.iterrows():
        text = row['text']
        selected_text = None

        start_char_idx = 0
        tweet_eg = TestTweetExample(
                text
        )
        tweet_eg.preprocess()
        tweet_examples.append(tweet_eg)
    print("Couldn't process",count,"points")
    return tweet_examples

In [ ]:
test_tweet_examples = create_tweet_examples_test(test)
x_test, _ = create_inputs_targets(test_tweet_examples)
print(f"{len(test_tweet_examples)} test points created.")

Couldn't process 0 points
[[  101   174  5242 ...     0     0     0]
 [  101  8050  6851 ...     0     0     0]
 [  101  7688  8556 ...     0     0     0]
 ...
 [  101  5952 21842 ...     0     0     0]
 [  101   180  7903 ...     0     0     0]
 [  101  1137 15796 ...     0     0     0]]
876 test points created.


In [ ]:
pred = model.predict(x_test)
np.array(pred).shape

(2, 876, 500)

In [ ]:
pred_start, pred_end = pred
count = 0

pred_text = []

test['prediction'] = np.zeros(len(test.text.values))

for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
    tweet_eg = test_tweet_examples[idx]
    if (tweet_eg.skip == True):
        pred_text.append("")
    else: 
        offsets = tweet_eg.text_token_to_char
        start = np.argmax(start)
        end = np.argmax(end)
        if start >= len(offsets):
            continue
        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_char_end = offsets[end][1]
            pred_ans = tweet_eg.text[pred_char_start:pred_char_end + 1]
        else:
            pred_ans = tweet_eg.text[pred_char_start:]

        # normalized_pred_ans = normalize_text(pred_ans)
        #print(normalized_pred_ans)
        pred_text.append(pred_ans)
        test['prediction'][idx] = pred_ans

test['start']=np.zeros(len(test.text.values))
test['end']=np.zeros(len(test.text.values))

for x in range(0,len(test)):
  test['start'][x] = test['text'][x].find(test['prediction'][x])
  test['end'][x] = start+len(test['prediction'][x])



lists = list()
for x in range(0,len(test)):
  a = np.array(range(int(test['start'][x]),int(test['end'][x])+1))
  lists.append(a)

test['spans_predict'] = lists

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

In [ ]:
test

,text,prediction,start,end,spans_predict
0,Evanukachum pondati kolandhainu sentiment irun...,"pondati kolandhainu sentiment irundha , apdiya...",12.0,110.0,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2..."
1,Adei ennada short film la irunthu suturukinga,Adei ennada short film la irunthu suturu,0.0,63.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,Super dialogue Oruthar mela visvasam kattrath...,visvasam kattrathukkaga innoruthavarai asinga,28.0,69.0,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,Ponnu padama edungada. India vallarasu agidun....,21.0,90.0,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 3..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,Kai mattum illa uyir,26.0,43.0,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 3..."
...,...,...,...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,punda passagala ungalu,99.0,45.0,[]
872,Kekka bekka short film mathri irukey,Kekka bekka short film mathr,0.0,51.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
873,Bayangaram... Trailerey ippadina appa Padam en...,Bayangaram... Trailerey ippadina appa Padam en...,0.0,91.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
874,komali rasini vesam pottu tamil ilichavayangal...,komali rasini vesam pottu tamil ilichavayangal...,0.0,77.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [ ]:
new_sub = pd.DataFrame({'text': test['text'], 'spans': test['spans_predict']})

new_sub.to_csv('drive/My Drive/CODE/SemVal/test_demo_detection.tsv',sep="\t", index=False)